In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import WalmartAmazonStructuredBenchmark

benchmark = WalmartAmazonStructuredBenchmark(data_dir_path="../data/")
benchmark

14:27:42 INFO:Extracting Walmart-Amazon-Structured...
14:27:42 INFO:Reading Walmart-Amazon-Structured row_dict...
14:27:43 INFO:Reading Walmart-Amazon-Structured train.csv...
14:27:43 INFO:Reading Walmart-Amazon-Structured valid.csv...
14:27:43 INFO:Reading Walmart-Amazon-Structured test.csv...


<WalmartAmazonStructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Walmart-Amazon/walmart_amazon_exp_data.zip

## Preprocess

In [5]:
attr_list = ['title', 'category', 'brand', 'modelno', 'price']

In [6]:
from tqdm.auto import tqdm
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_info_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'title_semantic': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'category': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'category_semantic': {
        'source_attr': 'category',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'brand': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'modelno': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
}

In [10]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=benchmark.row_dict)
row_numericalizer.attr_info_dict

14:27:43 INFO:For attr=title, computing actual max_str_len
14:27:43 INFO:actual_max_str_len=23 must be pair to enable NN pooling. Updating to 24
14:27:43 INFO:For attr=title, using actual_max_str_len=24
14:27:44 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
14:27:47 INFO:For attr=category, computing actual max_str_len
14:27:47 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
14:27:47 INFO:For attr=category, using actual_max_str_len=16
14:27:47 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
14:27:50 INFO:For attr=brand, computing actual max_str_len
14:27:50 INFO:For attr=brand, using actual_max_str_len=48
14:27:50 INFO:For attr=modelno, computing actual max_str_len
14:27:50 INFO:For attr=modelno, using actual_max_str_len=48
14:27:50 INFO:For attr=price, computing actual max_str_len
14:27:50 INFO:For attr=price, using actual_max_str_len=10


{'title': NumericalizeInfo(source_attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=24, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'title_semantic': NumericalizeInfo(source_attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 

In [11]:
datamodule = benchmark.build_pairwise_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=10,
    row_batch_size=16,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = f'f1-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

14:27:50 INFO:GPU available: True, used: True
14:27:50 INFO:TPU available: None, using: 0 TPU cores
14:27:50 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [14]:
trainer.fit(model, datamodule)

14:27:52 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 22.6 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
13.7 M    Trainable params
8.9 M     Non-trainable params
22.6 M    Total params


14:27:53 WARNING:true_pair_set has 38 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [15]:
model.blocker_net.get_signature_weights()

{'title': 0.2869834303855896,
 'title_semantic': 0.24623066186904907,
 'category': 0.05953275412321091,
 'category_semantic': 0.04035832732915878,
 'brand': 0.1752552092075348,
 'modelno': 0.14158503711223602,
 'price': 0.05005456134676933}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.42464246424642466,
 'valid_f1_at_0.5': 0.7901234567901234,
 'valid_f1_at_0.7': 0.9343832020997374,
 'valid_f1_at_0.9': 0.5703703703703703,
 'valid_pair_entity_ratio_at_0.3': 1.8891820580474934,
 'valid_pair_entity_ratio_at_0.5': 0.7730870712401056,
 'valid_pair_entity_ratio_at_0.7': 0.49604221635883905,
 'valid_pair_entity_ratio_at_0.9': 0.20316622691292877,
 'valid_precision_at_0.3': 0.26955307262569833,
 'valid_precision_at_0.5': 0.6552901023890785,
 'valid_precision_at_0.7': 0.9468085106382979,
 'valid_precision_at_0.9': 1.0,
 'valid_recall_at_0.3': 1.0,
 'valid_recall_at_0.5': 0.9948186528497409,
 'valid_recall_at_0.7': 0.9222797927461139,
 'valid_recall_at_0.9': 0.39896373056994816}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

[{'test_f1_at_0.3': 0.48614609571788414,
  'test_f1_at_0.5': 0.8584269662921349,
  'test_f1_at_0.7': 0.8999999999999999,
  'test_f1_at_0.9': 0.5210727969348659,
  'test_pair_entity_ratio_at_0.3': 1.5651041666666667,
  'test_pair_entity_ratio_at_0.5': 0.65625,
  'test_pair_entity_ratio_at_0.7': 0.4869791666666667,
  'test_pair_entity_ratio_at_0.9': 0.17708333333333334,
  'test_precision_at_0.3': 0.3211314475873544,
  'test_precision_at_0.5': 0.7579365079365079,
  'test_precision_at_0.7': 0.9144385026737968,
  'test_precision_at_0.9': 1.0,
  'test_recall_at_0.3': 1.0,
  'test_recall_at_0.5': 0.9896373056994818,
  'test_recall_at_0.7': 0.8860103626943006,
  'test_recall_at_0.9': 0.35233160621761656}]